In [2]:
import pandas as pd
from os import path
from ete3 import Tree, TreeStyle, NodeStyle

In [3]:
t = Tree("../data/gtt/gtt_cyanobacteria/gtt_cyanobacteria.tre")
t_ = Tree("../data/gtt/gtt_bacteria/gtt_bacteria.tre")

In [5]:
md = pd.read_csv("../data/tables/new_metadata_table.csv", sep=";")

In [8]:
nst0 = NodeStyle()
nst1 = NodeStyle()
nst1["bgcolor"] = "#D5E3D3" # verde
nst2 = NodeStyle()
nst2["bgcolor"] = "#F5D2D2" # vermelho
nst3 = NodeStyle()
nst3["bgcolor"] = "LightCyan" #"#DFF2F3" # azul
nst4 = NodeStyle()
nst4["bgcolor"] = "#ECD2EF" # roxo
nst5 = NodeStyle()
nst5["bgcolor"] = "#F2F1D4" # amarelo
nst6 = NodeStyle()
nst6["bgcolor"] = "#D1D1D1" # cinza


def make_branches_bigger(node, new_size):
    node.img_style["hz_line_width"] = new_size # Change the horizotal lines stroke size
    node.img_style["vt_line_width"] = new_size # Change the vertical lines stroke size
    for c in node.children:
        make_branches_bigger(c, new_size)

make_branches_bigger(t, 5)

for node in t.traverse():
    if node.is_leaf():
        node.name = node.name.replace("_", " ")
        if "nan" in node.name:
            print(node.name)
            node.delete()
        if "Pseudosynechococcus" in node.name:
            node.set_style(nst1)
        elif "Parasynechococcus" in node.name:
            node.set_style(nst2)
        elif "Regnicoccus" in node.name:
            node.set_style(nst3)
        elif "Magnicoccus" in node.name:
            node.set_style(nst4)
        elif "Inmanicoccus" in node.name:
            node.set_style(nst5)
        elif "Lacustricoccus" in node.name:
            node.set_style(nst6)
        elif "Synechospongium" in node.name:
            node.set_style(nst1)            
        elif "Synechococcus" in node.name:
            node.set_style(nst3)            
        elif "Enugrolinea" in node.name:
            node.set_style(nst2)    
        elif "Leptococcus" in node.name:
            node.set_style(nst4)                
        elif "Prochlorococcus" in node.name:
            t.set_outgroup(node)
            make_branches_bigger(node, 5)
            print(node.name)
            node.set_style(nst0)
        elif any((s in node.name for s in ("Coccusdissimilis", "Leptovivax", "Stenotopis", "Brevicoccus", "Lividococcus"))):
            print(node.name)
            node.delete()

Prochlorococcus marinus CCMP1375


In [9]:
c = TreeStyle()
c.mode = "r"
c.scale = 120
t.show(tree_style=c)